In [1]:
from ultralytics import YOLO
modelRes = YOLO("best.pt")

test model

In [2]:
results = modelRes.val(data="data.yaml")

# Print specific metrics
print("Model names: ", modelRes.names)
print("Class indices with average precision:", results.ap_class_index)
print("Average precision:", results.box.ap)
print("Average precision at IoU=0.50:", results.box.ap50)
print("Class indices for average precision:", results.box.ap_class_index)
print("F1 score:", results.box.f1)
print("Mean average precision:", results.box.map)
print("Mean average precision at IoU=0.50:", results.box.map50)
print("Mean average precision at IoU=0.75:", results.box.map75)
print("Mean average precision for different IoU thresholds:", results.box.maps)
print("Mean precision:", results.box.mp)
print("Mean recall:", results.box.mr)
print("Precision:", results.box.p)
print("Recall:", results.box.r)

Ultralytics 8.3.18  Python-3.11.0 torch-2.5.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
YOLOv10s summary (fused): 293 layers, 8,037,282 parameters, 0 gradients, 24.5 GFLOPs


val: Scanning C:\Users\PC\Desktop\thesis\detection\data\valid\labels.cache... 486 images, 0 backgrounds, 0 corrupt: 100%|██████████| 486/486 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:05<00:00,  6.13it/s]


                   all        486        791      0.846      0.695      0.789      0.547
               Gernade        103        161      0.869      0.584      0.726       0.53
                   Gun        204        313      0.923      0.771      0.886      0.618
                 Knife        268        317      0.744      0.729      0.754      0.494
Speed: 0.5ms preprocess, 6.6ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs\detect\val
Model names:  {0: 'Gernade', 1: 'Gun', 2: 'Knife'}
Class indices with average precision: [0 1 2]
Average precision: [    0.52994     0.61777     0.49385]
Average precision at IoU=0.50: [    0.72623     0.88637     0.75361]
Class indices for average precision: [0 1 2]
F1 score: [    0.69859     0.84054     0.73647]
Mean average precision: 0.5471883298647278
Mean average precision at IoU=0.50: 0.7887397781970646
Mean average precision at IoU=0.75: 0.6366083364985254
Mean average precision for different IoU thresholds: [    0.

In [2]:
import cv2
import os
import numpy as np


# Auto-configure CLAHE parameters based on the image
def enhance_contrast_auto(gray_image):
    # Get image dimensions
    height, width = gray_image.shape

    # Dynamically set the CLAHE parameters
    # Clip limit adapts based on image intensity distribution
    clip_limit = max(
        2.0, min(4.0, gray_image.std() / 20)
    )  # Adjusted based on image contrast
    # Tile grid size adapts based on image size
    grid_size = (
        (8, 8) if max(height, width) <= 1000 else (16, 16)
    )  # Larger for high-res

    # Create CLAHE with dynamic parameters
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=grid_size)

    # Apply CLAHE to enhance contrast
    enhanced_image = clahe.apply(gray_image)
    return enhanced_image


def reduce_noise_auto(image):
    """
    Applies a Bilateral Filter with adaptive parameters based on the input grayscale image.

    :param image: Grayscale image (numpy array).
    :return: Filtered image.
    """
    # Calculate noise level (standard deviation)
    noise_std = np.std(image)

    # Image dimensions
    height, width = image.shape
    image_size = max(height, width)

    # Adaptive parameters
    d = max(5, image_size // 100)  # Neighborhood diameter, scaled by image size
    sigma_color = noise_std * 10  # Adjust intensity influence based on noise level
    sigma_space = d * 1.5  # Spatial influence, linked to diameter

    # Apply Bilateral Filter
    filtered_image = cv2.bilateralFilter(image, d, sigma_color, sigma_space)

    return filtered_image


def auto_edge_enhance(gray_image):
    # Calculate the image contrast (standard deviation of pixel values)
    contrast = gray_image.std()

    # Set a base clip limit and tile grid size
    clip_limit = 2.0  # default
    tile_grid_size = (8, 8)  # default size

    # Adjust clip limit based on image contrast (higher contrast = higher clip limit)
    if contrast > 50:
        clip_limit = 3.0
    elif contrast > 30:
        clip_limit = 2.5
    else:
        clip_limit = 2.0

    # Adjust tile grid size based on image resolution (smaller images = smaller tiles)
    height, width = gray_image.shape
    if height < 500 or width < 500:
        tile_grid_size = (4, 4)
    elif height < 1000 or width < 1000:
        tile_grid_size = (8, 8)
    else:
        tile_grid_size = (16, 16)

    # Apply CLAHE with the auto-configured parameters
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    clahe_img = clahe.apply(gray_image)

    return clahe_img


def dehaze_auto(gray_image):
    # Calculate dynamic CLAHE parameters based on image properties
    mean_brightness = gray_image.mean()
    contrast_std = gray_image.std()

    # Dynamically configure CLAHE
    clip_limit = max(
        2.0, min(4.0, contrast_std / 10)
    )  # Adjust clip limit based on contrast
    grid_size = (8, 8) if gray_image.shape[0] <= 1000 else (16, 16)  # Adjust grid size

    # Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=grid_size)
    enhanced_image = clahe.apply(gray_image)

    # Normalize intensity if brightness is too low
    if mean_brightness < 100:  # If the image is very dark
        normalized_image = cv2.normalize(enhanced_image, None, 0, 255, cv2.NORM_MINMAX)
        return normalized_image
    else:
        return enhanced_image


def adaptive_unsharp_masking(gray_image):
    """
    Automatically sharpens a grayscale image using unsharp masking with dynamic adjustments
    based on image size, brightness, and contrast.

    Args:
        gray_image (numpy.ndarray): Input grayscale image (loaded with cv2).

    Returns:
        sharpened_image (numpy.ndarray): The sharpened grayscale image.
    """
    # Get image dimensions
    height, width = gray_image.shape

    # Dynamically adjust blur kernel size based on image dimensions
    # Larger images get larger kernels
    kernel_size = (max(3, width // 500), max(3, height // 500))

    # Apply Gaussian blur
    blurred_image = cv2.GaussianBlur(gray_image, kernel_size, 0)

    # Calculate the mean and standard deviation of the image
    mean_intensity = np.mean(gray_image)
    std_intensity = np.std(gray_image)

    # Dynamically set alpha and beta
    # Higher contrast (std_intensity) reduces alpha (to avoid over-sharpening)
    # Lower brightness increases beta (to enhance edges more aggressively)
    alpha = 1.0 + (std_intensity / 128)  # Base sharpness control
    beta = -0.5 - (mean_intensity / 255)  # Edge control based on brightness

    # Combine the original image and the blurred image for sharpening
    sharpened_image = cv2.addWeighted(gray_image, alpha, blurred_image, beta, 0)

    return sharpened_image

In [3]:
import cv2
import numpy as np

# Read the PNG image
png_image = cv2.imread('testKnife.jpg', cv2.IMREAD_GRAYSCALE)

# resized_image = cv2.resize(png_image, (640, 640))
denoised_image = reduce_noise_auto(png_image)
enhance_contract_image = enhance_contrast_auto(denoised_image)
unsharp_masking_image = adaptive_unsharp_masking(enhance_contract_image)

input_image = cv2.merge([unsharp_masking_image, unsharp_masking_image, unsharp_masking_image])

results = modelRes.predict(input_image,conf=0.30,)
results[0].show()
results[0].boxes


0: 544x640 1 Knife, 134.2ms
Speed: 8.0ms preprocess, 134.2ms inference, 50.4ms postprocess per image at shape (1, 3, 544, 640)


ultralytics.engine.results.Boxes object with attributes:

cls: tensor([2.], device='cuda:0')
conf: tensor([0.8434], device='cuda:0')
data: tensor([[ 48.7240,  81.2650, 640.0000, 528.8535,   0.8434,   2.0000]], device='cuda:0')
id: None
is_track: False
orig_shape: (536, 640)
shape: torch.Size([1, 6])
xywh: tensor([[344.3620, 305.0592, 591.2760, 447.5884]], device='cuda:0')
xywhn: tensor([[0.5381, 0.5691, 0.9239, 0.8351]], device='cuda:0')
xyxy: tensor([[ 48.7240,  81.2650, 640.0000, 528.8535]], device='cuda:0')
xyxyn: tensor([[0.0761, 0.1516, 1.0000, 0.9867]], device='cuda:0')